In [295]:
! pip3 install pandas pycoingecko 
import pandas as pd
import datetime as dt
import time
import json
from pycoingecko import CoinGeckoAPI
from tqdm import tqdm
import requests
cg = CoinGeckoAPI()

In [411]:
tweet_dic={
    "id": ['1435120030245662722','1354488469091975168','1433401141237256193','1392602041025843203','1356310219215699968','1340590280848908288'],
    "coin": ['bitcoin','bitcoin','bitcoin','bitcoin','bitcoin','dogecoin'],
    "currency": ['usd','usd','usd','usd','usd','usd'],
    "creator": ['Nayib Bukele','Mayor Francis Suarez','Michael Saylor','Elon Musk','Willy Woo','Elon Musk'],
}
tweet_df = pd.DataFrame(tweet_dic)

In [412]:
#tweet_date = "2018-07-18 01:58:00"
#coin='bitcoin'
#currency='usd'
#date_format = "%Y-%m-%d %H:%M:%S"
#1 day from current time will produce 5min interval
#1-90 hourly interval
#above 90 days is daily
#number_of_days = 45
#number_of_hours = 0
#tweet_datetime = pd.to_datetime(tweet_date,format=date_format)
#tweet_datetime= pd.to_datetime("2021-09-07T05:57:00")

In [413]:
# To set your enviornment variables in your terminal run the following line:
#export 'BEARER_TOKEN'='AAAAAAAAAAAAAAAAAAAAACGakwEAAAAA9AmAwPyPkNa%2BDAFSRniHzrezwiY%3Dz2T8TZvftexbR58cnUQ6wLip4ra9SfafHlJaUOmLn8lv5Fah5Z'
bearer_token ='AAAAAAAAAAAAAAAAAAAAACGakwEAAAAA9AmAwPyPkNa%2BDAFSRniHzrezwiY%3Dz2T8TZvftexbR58cnUQ6wLip4ra9SfafHlJaUOmLn8lv5Fah5Z'
#bearer_token = os.environ.get("AAAAAAAAAAAAAAAAAAAAACGakwEAAAAA9AmAwPyPkNa%2BDAFSRniHzrezwiY%3Dz2T8TZvftexbR58cnUQ6wLip4ra9SfafHlJaUOmLn8lv5Fah5Z")


def create_url(twt_id):
    tweet_fields = "tweet.fields=author_id,created_at,text"
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    ids = "ids=" + twt_id
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    url = "https://api.twitter.com/2/tweets?{}&{}".format(ids, tweet_fields)
    return url


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2TweetLookupPython"
    return r


def connect_to_endpoint(url):
    response = requests.request("GET", url, auth=bearer_oauth)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def get_tweet_timestamp(twt_id):
    url = create_url(twt_id)
    json_response = connect_to_endpoint(url)
    #return timestamp with timezone removed
    return pd.to_datetime(json_response['data'][0]['created_at'][:-1])

In [414]:
tweet_df['timestamp']=tweet_df['id'].map(lambda ids: get_tweet_timestamp(ids))

200
200
200
200
200
200


In [415]:
tweet_df.to_csv("./tweets.csv",index=False)

In [416]:
tweets = pd.read_csv('./tweets.csv')

In [417]:
tweets['timestamp']=tweets['timestamp'].astype('datetime64[ns]')

In [418]:
def generate_prices_df(twt_dt,coin,currency,number_of_days=45,number_of_hours=0):
    tweet_from_timestamp = (twt_dt - dt.timedelta(days=number_of_days,hours=number_of_hours)).timestamp()
    tweet_to_timestamp = (twt_dt + dt.timedelta(days=number_of_days,hours=number_of_hours)).timestamp()
    coin_gecko_data = cg.get_coin_market_chart_range_by_id(id=coin,
                                                vs_currency=currency,
                                                from_timestamp=tweet_from_timestamp,
                                                to_timestamp=tweet_to_timestamp)
     
    return parse_coin_data(coin_gecko_data)
def prepend_past_prices(df,coin,currency,number_of_days=45,n=1):
    copy_df = df.copy()
    for i in range(n):
        first_date = list(copy_df['date'])[0] 
        to_timestamp = (first_date - dt.timedelta(hours=0.5)).timestamp()
        from_timestamp = (first_date - dt.timedelta(days=(number_of_days * 2))).timestamp()
        coin_gecko_data = cg.get_coin_market_chart_range_by_id(id=coin,
                                                               vs_currency=currency,
                                                               from_timestamp=from_timestamp,
                                                               to_timestamp=to_timestamp)
        new_df = parse_coin_data(coin_gecko_data)
        copy_df = pd.concat([new_df,copy_df]).reset_index(drop=True)
    return copy_df
def append_future_prices(df,coin,currency,number_of_days=45,n=1):
    copy_df = df.copy()
    for i in range(n):
        last_date = list(copy_df['date'])[-1] 
        from_timestamp = (last_date + dt.timedelta(hours=0.5)).timestamp()
        to_timestamp = (last_date + dt.timedelta(days=(number_of_days*2))).timestamp()
        coin_gecko_data = cg.get_coin_market_chart_range_by_id(id=coin,
                                                               vs_currency=currency,
                                                               from_timestamp=from_timestamp,
                                                               to_timestamp=to_timestamp)
        new_df = parse_coin_data(coin_gecko_data)
        copy_df = pd.concat([copy_df,new_df]).reset_index(drop=True)
    return copy_df
def get_large_intervals(twt_dt,coin,currency,number_of_days=45,n=1):
    original_df = generate_prices_df(twt_dt,coin,currency,number_of_days)
    past = prepend_past_prices(original_df,coin,currency,number_of_days,n)
    return append_future_prices(past,coin,currency,number_of_days,n)

In [431]:
def parse_coin_data(coin_data):
    parsed_data = {}
    timestamps=list(map(lambda x:x[0],coin_data['prices']))
    for key in coin_data.keys():
        parsed_data[key] = list(map(lambda x:x[1], coin_data[key]))
    parsed_data['timestamps']=timestamps
    df=pd.DataFrame(parsed_data)
    df['date']=pd.to_datetime(df['timestamps'],origin='unix',unit='ms')
    return df
def produce_intervals_for_each(tweets_df,size=1,directory="./prices/"):
    for _, row in tqdm (tweets_df.iterrows(), desc="Creating intervals...") :
        coin = row['coin']
        currency = row['currency']
        creator = row['creator']
        timestamp = row['timestamp']
        row_df = get_large_intervals(timestamp,coin,currency,n=size)
        filepath = directory + (creator.lower() + " " + coin + " vs " + currency + " " + str(timestamp) + " " + ".csv").replace(" ", "_")
        row_df.to_csv(filepath,index=False)

In [432]:
produce_intervals_for_each(tweets,size=3)

Creating intervals...: 6it [03:30, 35.11s/it]


In [433]:
last =tweets[tweets['coin']=='dogecoin']
produce_intervals_for_each(last,size=3)

Creating intervals...: 1it [00:00,  1.31it/s]
